In [2]:
import pandas as pd
import numpy as np
from numpy import loadtxt
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [17]:
def subids(file):
    file_subid=pd.read_csv(file)
    lista=[]
    count=0
    for x in range(len(file_subid['paciente'])):
        ids=file_subid['paciente'][x]
        if ids not in lista:
            lista.append(ids)
            count+=1
    return lista

lista=subids("0-DS1_metascans.csv")

for i in range(len(lista)):
    lista[i].replace("''","")
print(len(lista))

426


In [29]:
#see all the radiomics features
def feature_analysis(file):
    radiomics_features=pd.read_csv(file)
    #radiomics_features.isna().values.sum()
    #radiomics_features=radiomics_features.dropna()
    cols = radiomics_features.columns
    #print(radiomics_features.select_dtypes(include=['category']).columns.tolist())
    #print(radiomics_features.select_dtypes(exclude=['category']).columns.tolist())
    #num_cols = radiomics_features._get_numeric_data().columns
    #print(list(set(cols) - set(num_cols)))
    #columns_to_drop = ['diagnostics_Image-original_Minimum','diagnostics_Image-original_Mean','diagnostics_Image-original_Maximum']
    #radiomics_features = radiomics_features.drop(columns_to_drop, axis = 1)
    #print(num_cols)
    print(radiomics_features.shape)
    return radiomics_features

file="A.all_features_scans.csv"
radiomics_features=feature_analysis(file)

(852, 2004)


In [31]:
#drop the columns of all features that aren't numeric
def drop_columns(radiomics_features):
    my_type = 'float64'

    dtypes = radiomics_features.dtypes.to_dict()
    
    for col_nam, typ in dtypes.items():

         if (typ != my_type) and col_nam != "ID":
                #print(col_nam)
                radiomics_features.drop(col_nam, axis = 1,inplace = True)
                
    #radiomics_features.drop("diagnostics_Image-original_Mean", axis = 1,inplace = True)
    #radiomics_features.drop("diagnostics_Image-original_Maximum", axis = 1,inplace = True)
    print(radiomics_features.shape)
    return radiomics_features
cleaned_features=drop_columns(radiomics_features)

#get a csv file with all the numeric radiomics features
dtypes = cleaned_features.dtypes.to_dict()
for col_nam, typ in dtypes.items():
        count=0
        num = cleaned_features[col_nam][0]
        for x in range(1,len(cleaned_features[col_nam])):
            if cleaned_features[col_nam][x]!=num:
                count+=1
        
        if count==0:
            cleaned_features.drop(col_nam, axis = 1,inplace = True)
        
        
print(cleaned_features.shape)
   
cleaned_features.to_csv("A.all_features_scans.csv", index=False)

(852, 2004)
(852, 2004)


In [32]:
ficheiro = pd.read_csv("A.all_features_scans.csv")
ficheiroUmaLinha = pd.read_csv("0-DS1_metascans.csv")
ages = []
mmse = []
cdr  = []
sex  = []
for index, row in ficheiroUmaLinha.iterrows():
    for colunaDois in ficheiro.columns:
        ages.append(row["idade_0"])
        ages.append(row["idade_24"])
        mmse.append(row["MMSE_0"])
        mmse.append(row["MMSE_24"])
        cdr.append(row["CDR_0"])
        cdr.append(row["CDR_24"])
        if row["sexo"]=="F":
            sex.append(0)
            sex.append(0)
        else:
            sex.append(1)
            sex.append(1)
        break
        
ficheiro["AGE"] = ages
ficheiro["MMSE"] = mmse
ficheiro["CDR"] = cdr
ficheiro["SEX"] = sex

print(ficheiro.shape)

ficheiro.to_csv("A1.cleaned_features_scans.csv", index=False)

(852, 2008)


In [46]:
def classificacao(MMSE,CDR):
        
    if MMSE > 24 and MMSE <=30 and CDR == 0.0:
        return "CN"
        
    if MMSE > 24 and MMSE <=30 and CDR == 0.5:
        return "MCI"
    
    if MMSE > 1 and MMSE <=24 and CDR == 0.5:
         return "AD"
        
    if MMSE > 1 and MMSE <=30 and CDR > 0.5:
        return "AD"
    
    else:
        return null

In [33]:
ficheiro = pd.read_csv("A1.cleaned_features_scans.csv")

pacientes = {}
paciente = ""
estadoAnterior = ""
for index, row in ficheiro.iterrows():
    mmse = row["MMSE"]
    cdr = row["CDR"]
    paciente_atual = row["ID"].split("/")[1][5:15]
    if paciente == paciente_atual:
        est = classificacao(mmse, cdr)
        estadoAnterior = estadoAnterior + "-" + est
        pacientes[paciente_atual] = estadoAnterior
    else:
        estadoAnterior = classificacao(mmse, cdr)
        paciente = paciente_atual
        
status =[]
for index, row in ficheiro.iterrows():
    paciente_atual = row["ID"].split("/")[1][5:15]
    status.append(pacientes[paciente_atual])

ficheiro["Transition"] = status

print(ficheiro.shape)

ficheiro.to_csv("A1.cleaned_features_scans.csv", index=False)

(852, 2009)


In [59]:
#labeling

ficheiro = pd.read_csv("A1.cleaned_features_scans.csv")

classes = ["CN-MCI", "CN-AD", "MCI-AD", "CN-CN", "MCI-MCI", "AD-AD"]
classes_ids = {"CN-MCI":0, "CN-AD":1, "MCI-AD":2, "CN-CN":3, "MCI-MCI":4, "AD-AD":5}
Transition_ID = []

for index, row in ficheiro.iterrows():
    if row["Transition"] not in classes:
        ficheiro.drop(index, inplace=True)
    else:
        Transition_ID.append(classes_ids[row["Transition"]])

ficheiro["Transition_ID"] = Transition_ID

print(ficheiro.shape)

ficheiro.to_csv("A1.cleaned_features_scans.csv", index=False)

(812, 2010)


In [70]:
#DELETE MONTH 24 IN THE WHOLE DATASET

ficheiro = pd.read_csv("A1.cleaned_features_scans.csv")

df = ficheiro.drop_duplicates(subset='ID', keep='first')
    
print(df.shape)
df.to_csv("A1.cleaned_features_scans.csv", index=False)

(406, 2010)


In [85]:
#COLUMN CLASS

ficheiro = pd.read_csv("A1.cleaned_features_scans.csv")

classe_lista = []
transit = list(ficheiro["Transition"])

for i in transit:
    a=i.split("-")
    classe_lista.append(a[0])

ficheiro["Class"] = classe_lista

print(ficheiro.shape)

ficheiro.to_csv("A1.cleaned_features_scans.csv", index=False)

(406, 2011)


In [87]:
#COLUMN CLASS_ID

ficheiro = pd.read_csv("A1.cleaned_features_scans.csv")

classes = ["CN", "MCI","AD"]
classes_ids = {"CN":0, "MCI":1, "AD":2}
Class_ID = []

for index, row in ficheiro.iterrows():
    if row["Class"] not in classes:
        ficheiro.drop(index, inplace=True)
    else:
        Class_ID.append(classes_ids[row["Class"]])

ficheiro["Class_ID"] = Class_ID

print(ficheiro.shape)

ficheiro.to_csv("A1.cleaned_features_scans.csv", index=False)

(406, 2012)


### Counts

In [90]:
def contagens_transicoes(ficheiro):
    contas = {"CN-MCI":0, "CN-AD":0, "MCI-AD":0, "CN-CN":0, "MCI-MCI":0, "AD-AD":0}
    for index, row in ficheiro.iterrows():
        contas[row["Transition"]] += 1
    return contas

contagens_transicoes(pd.read_csv("A1.cleaned_features_scans.csv"))

{'CN-MCI': 16,
 'CN-AD': 1,
 'MCI-AD': 88,
 'CN-CN': 125,
 'MCI-MCI': 101,
 'AD-AD': 75}

In [92]:
def contagens_classes(ficheiro):
    contas = {"CN":0, "MCI":0, "AD":0}
    for index, row in ficheiro.iterrows():
        contas[row["Class"]] += 1
    return contas

contagens_classes(pd.read_csv("A1.cleaned_features_scans.csv"))

{'CN': 142, 'MCI': 189, 'AD': 75}

###  EXTRA-PROCESSING:

In [45]:
# PUT THE ID CORRECTLY - ONLY WITH THE PATIENT NUMBER

ficheiroBase = pd.read_csv("A1.cleaned_features_scans.csv")

for index, row in ficheiroBase.iterrows():
    idBase=row[0]
    idBase=str(idBase).split("/")[1]
    idBaseF=idBase.split("_")[1]+"_"+idBase.split("_")[2]+"_"+idBase.split("_")[3]

    ficheiroBase["ID"].replace(row[0], idBaseF, inplace = True)
    
print(ficheiroBase.shape)

ficheiroBase.to_csv("A1.cleaned_features_scans.csv", index = False)

(812, 2010)


In [100]:
#normalize the data

file = "A1.cleaned_features_scans.csv"

csv = pd.read_csv(file)

columns=["ID","SEX","AGE","MMSE","CDR","Transition","Transition_ID","Class","Class_ID"]

def normalize_radiomic():
    
    ficheiro = pd.read_csv(file)
    data = ficheiro.drop(columns=columns)
    data_normalized = data.apply(lambda x:( (x - x.min()) / (x.max()-x.min())))
    return data_normalized
   
df=normalize_radiomic()

listaID=csv["ID"]
listaSex=csv["SEX"]
listaAge=csv["AGE"]
listaMMSE=csv["MMSE"]
listaCDR=csv["CDR"]
listaTransitions=csv["Transition"]
listaTransitionsID=csv["Transition_ID"]
listaClasses=csv["Class"]
listaClassesID=csv["Class_ID"]

df.insert(0, 'ID', listaID)
df.insert(1, 'Sex', listaSex)
df.insert(2, 'Age', listaAge)
df.insert(3, 'MMSE', listaMMSE)
df.insert(4, 'CDR', listaCDR)
df.insert(5, 'Transition_Label', listaTransitionsID)
df.insert(6, 'Transition', listaTransitions)
df.insert(7, 'Class_Label', listaClassesID)
df.insert(8, 'Class', listaClasses)

print(df.shape)    
df.to_csv("A2.features_final_scans_normalized.csv", index = False)

(406, 2012)


### POS-PROCESSING (remove labels that do not exist)

In [103]:
def contagens_transicoes(ficheiro):
    contas = {"CN-MCI":0, "CN-AD":0, "MCI-AD":0, "CN-CN":0, "MCI-MCI":0, "AD-AD":0}
    for index, row in ficheiro.iterrows():
        contas[row["Transition"]] += 1
    return contas

contagens_transicoes(pd.read_csv("A2.features_final_scans_normalized.csv"))

{'CN-MCI': 16,
 'CN-AD': 1,
 'MCI-AD': 88,
 'CN-CN': 125,
 'MCI-MCI': 101,
 'AD-AD': 75}

In [104]:
#Since there is only 1 CN-AD case, it will be removed.

ficheiro = pd.read_csv("A2.features_final_scans_normalized.csv")

classes = ["CN-MCI", "MCI-AD", "CN-CN", "MCI-MCI", "AD-AD"]
classes_ids = {"CN-MCI":0, "MCI-AD":1, "CN-CN":2, "MCI-MCI":3, "AD-AD":4}
Transition_ID = []

for index, row in ficheiro.iterrows():
    if row["Transition"] not in classes:
        ficheiro.drop(index, inplace=True)
    else:
        Transition_ID.append(classes_ids[row["Transition"]])

ficheiro["Transition_Label"] = Transition_ID

print(ficheiro.shape)

ficheiro.to_csv("A3.DS_Brain.csv", index=False)

(405, 2013)


In [12]:
df=pd.read_csv("A3.DS_Brain.csv")
#df["Transition_Label"].value_counts()

df["Transition_Label"].replace(2,1, inplace = True)
df["Transition_Label"].replace(3,2, inplace = True)
df["Transition_Label"].replace(4,3, inplace = True)
df["Transition_Label"].replace(5,4, inplace = True)

df["Transition_Label"].value_counts()

df.to_csv("A3.DS_Brain.csv", index=False)

In [11]:
def contagens_transicoes(ficheiro):
    contas = {"CN-MCI":0, "CN-AD":0, "MCI-AD":0, "CN-CN":0, "MCI-MCI":0, "AD-AD":0}
    for index, row in ficheiro.iterrows():
        contas[row["Transition"]] += 1
    return contas

contagens_transicoes(pd.read_csv("A3.DS_Brain.csv"))

{'CN-MCI': 16,
 'CN-AD': 0,
 'MCI-AD': 88,
 'CN-CN': 125,
 'MCI-MCI': 101,
 'AD-AD': 75}

In [3]:
# DELETE EXTRA COLUMN

df=pd.read_csv("A3.DS_Brain.csv")
print(df.shape)
df.drop("Transition_ID",axis=1,inplace=True)
print(df.shape)
df.to_csv("A3.DS_Brain.csv", index=False)

(405, 2013)
(405, 2012)
